In [7]:
# import modules
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely import wkt
from shapely.geometry import Point
import shapely
import osmnx as ox
import pandas as pd
import networkx as nx
from shapely.geometry import LineString
from tabulate import tabulate
import math as math
from geojson import Feature, LineString, FeatureCollection



In [2]:
!which python

/home/kriso2/anaconda3/envs/geo_env/bin/python


In [8]:
# Reading csv files into panda DataFrames
dfNodes = pd.read_csv("/mnt/c/users/kriso/documents/itu/5th_semester/research_project/CoolWalk/data/CoolWalksDataStorge/nodes.csv")
dfEdges = pd.read_csv("/mnt/c/users/kriso/documents/itu/5th_semester/research_project/CoolWalk/data/CoolWalksDataStorge/edges.csv")

In [4]:
#Testing stuff. Not necessary to execute
geopint = dfNodes['geopoint']
print(geopint)

0        POINT (-1.0771582 53.0405631)
1        POINT (-1.1072849 52.9308475)
2        POINT (-1.2230846 52.9640918)
3         POINT (-1.1282842 53.010786)
4        POINT (-1.1900777 52.9092031)
                     ...              
25643                              NaN
25644                              NaN
25645                              NaN
25646                              NaN
25647                              NaN
Name: geopoint, Length: 25648, dtype: object


In [6]:
#Testing stuff. Not necessary to execute
x = gpd.GeoSeries.from_wkt(dfNodes['geopoint']) #This exact function (with the exact same input) does not work for some reason when run in python file. (It ran several times previously without problems...)


0        POINT (-1.07716 53.04056)
1        POINT (-1.10728 52.93085)
2        POINT (-1.22308 52.96409)
3        POINT (-1.12828 53.01079)
4        POINT (-1.19008 52.90920)
                   ...            
25643                         None
25644                         None
25645                         None
25646                         None
25647                         None
Length: 25648, dtype: geometry

In [ ]:
#Testing stuff. Not necessary to execute
#creating the shadow fraction column separately
shadow_fraction_column = (dfEdges['shadowed_length'] / dfEdges['full_length'])
#print(shadow_fraction_column.notnull())

In [9]:
#creating shadow fraction column in the Edges dataframe
dfEdges['shadow_fraction'] = (dfEdges['shadowed_length'] / dfEdges['full_length'])

In [10]:
#Testing. Not necessary to run
# displaying shadow_fraction column of the new Edges dataframe, and checking that they are not all NaN. Display the new shadow_fraction column consisting only of the rows where its value os not na
shadow_fraction_column = dfEdges.loc[dfEdges['shadow_fraction'].notnull()]['shadow_fraction']

#print(shadow_fraction_column)
#print(shadow_fraction_column.dtypes)

In [ ]:
#Just a check. Not necessary to run

# Print Nodes dataframe with helper nodes. Prints the nodes dataframe consisting only of rows where value of osm_id column == 0
helper_nodes = dfNodes.loc[dfNodes['osm_id'] == 0]

#print(tabulate(helper_nodes, headers='keys', tablefmt='psql'))


In [ ]:
##Just a check. Not necessary to run 

# Print Edges dataframe with helper edges. Prints the edges dataframe consisting only of rows where value of helper column ==True
helper_edges = dfEdges.loc[dfEdges['helper'] == True]

#print(tabulate(helper_edges, headers='keys', tablefmt='psql'))

In [ ]:
##Testing

# Find and display edges with NaN-valued 'shadowed_part_length' row instances. Displaying the shadowed_part_length column, consisting only of the rows where its value os IS na/null
shadowed_part_length_column_NaNs = dfEdges.loc[dfEdges['shadowed_part_length'].isna()]['shadowed_part_length']

# Find and displaying shadowed_part_length column of the new Edges dataframe, and checking that they are not all NaN. Display the shadowed_part_length column consisting only of the rows where its value os NOT na/null
shadowed_part_length_column_not_NaNs = dfEdges.loc[dfEdges['shadowed_part_length'].notnull()]['shadowed_part_length']

#print(shadowed_part_length_column_NaNs)

In [ ]:
##Testing

# Find and display edges with NaN-valued 'full_length' row instances. Displaying the full_length column, consisting only of the rows where its value os IS na/null
full_length_column_NaNs = dfEdges.loc[dfEdges['full_length'].isna()]['full_length']

full_length_column_not_NaNs = dfEdges.loc[dfEdges['full_length'].notnull()]['full_length']

print(full_length_column_NaNs)

In [96]:
##Testing 

# Display Get the average shadowed_part_length value, to replace missing shadowed_part_length values
average_shadowed_part_length = dfEdges['shadowed_part_length'].mean()

print(average_shadowed_part_length)

25.679229294813386


In [ ]:
#Dont execute

#TODO If shadow is None it is 0. However, our current cost-function is not compatible with this, since it is a division, and will thus result in division by 0 errors.
#   TEMPORARY SOLUTION: Dont execute this cell. Let the shortest_path_algorithm cost-function fill out the None shadow values with 1. 

#Find and replace all None-valued 'shadowed_part_length' instances in edges with 0
dfEdges['shadowed_part_length'] = dfEdges['shadowed_part_length'].fillna(0)

In [11]:
# Find and replace all None-valued 'full_length' instances in edges with the average full_length value
dfEdges['full_length'] = dfEdges['full_length'].fillna(dfEdges['full_length'].mean())

In [ ]:
#TESTING. Not necessary to execute
    # Checking that helper node with vertex_id=25262 can be found in the nodes before removing helpers
    #print(dfNodes.loc[dfNodes['vertex_id'] == 25262])

In [12]:
# Find and discard helper nodes from dfNodes. Helper nodes do not have a 'geopoint' attribute value in the dataframe
dfNodes.drop(dfNodes[dfNodes.osm_id == 0].index, inplace=True)
#print(tabulate(gdfNodes, headers='keys', tablefmt='psql'))

In [ ]:
#TESTING. Not necessary to execute
    # Checking that helper node with vertex_id=25262 has been removed 
    # print(dfNodes.loc[dfNodes['vertex_id'] == 25262])

In [13]:
# Find and discard helper edges from gdfEdges
dfEdges.drop(dfEdges[dfEdges.helper == True].index, inplace=True)
#print(tabulate(gdfEdges, headers='keys', tablefmt='psql'))

In [14]:
# Setting osmid as index for nodes (mutating the dataframe). ATTENTION! the osmnx code will create the x_lookup and y_lookup dictionaries based on the index of the nodes data frame. That is, the keys of these dictionaries will consist
# of the values of the index column, and the values will be the lon(x_lookup) and lat(y_lookup) values respectively   

dfNodes.set_index('vertex_id', inplace=True) 
#dfNodes.set_index('osm_id', inplace=True) 



#Printing dataframes in readable format
#print(tabulate(gdfNodes, headers='keys', tablefmt='psql'))

In [15]:
#setting src_id, dst_id and osm_id of edge geodf´s as the multiindex(u, v, key) of each node (mutating the dataframe)
dfEdges.set_index(['src_id','dst_id','osm_id'], inplace=True) #should we use the osm_id from the Nodes instead, since Henrik says they have 0 values to indicate multi-edges, which is what the value 'k' is intended for in the documentation of utils_graph.graph_from_gdfs

#Printing dataframes in readable format
#print(tabulate(gdfEdges, headers='keys', tablefmt='psql'))

In [16]:
# Transforming relevant columns of DataFrames into GeoSeries to be used as geometry column of GeoDataFrames
dfNodes['geopoint'] = gpd.GeoSeries.from_wkt(dfNodes['geopoint'])
dfEdges['edgegeom'] = gpd.GeoSeries.from_wkt(dfEdges['edgegeom'])

In [17]:
# Defining GeoDataFrames with GeoSeries columns from above
gdfNodes = gpd.GeoDataFrame(dfNodes, geometry='geopoint', crs="EPSG:27700")
gdfEdges = gpd.GeoDataFrame(dfEdges, geometry='edgegeom', crs="EPSG:27700")

In [18]:
# Data wrangling: Editing/andjustment GeoDataFrames
gdfNodes.rename(columns = {'lat':'x', 'lon':'y'}, inplace = True)
gdfNodes["x"] = gdfNodes["geopoint"].x
gdfNodes["y"] = gdfNodes["geopoint"].y

#print(tabulate(gdfNodes, headers='keys', tablefmt='psql'))



In [19]:
G = ox.utils_graph.graph_from_gdfs(gdfNodes, gdfEdges, graph_attrs=None)

In [ ]:
G.is_multigraph()

In [79]:
def weigth_function_original(u, v, d):
    assert len(list(attr.get('full_length', 1) for attr in d.values())) == 1 #the graph only have one sttribute with name = 'full_length'
    dict = list(d.values())[0].get('full_length')  # w could perhaps also just access full_length and shadowed_length in this way?
    if dict is not None : #apparently some edges does not have a 'full_length' attribute, and in that case the attribute value will be none. check in the data frame for none values
        print(dict)
    full_length = min(attr.get('full_length', 1) for attr in d.values()) #if 'full_length' value is None, then it will default to 1... keyError will return None because we use the get() method
    shadow_length = min(attr.get('shadowed_length', 1) for attr in d.values())
    result = full_length/shadow_length # we get many 1´s, which either because they default to 1, or because shadow_length==full_length(very unlikely). So in the next line, we filter out all the 1´s to make sure that we actually have real meaningfull values
    if not math.isclose(result, 1): #
        print(result)
    return result

In [88]:
def weight_function_1(u, v, d):
    #print(d.values())
    assert len(list(attr.get('full_length', 0) for attr in d.values())) == 1 #the graph only have one sttribute with name = 'full_length'
    dict = list(d.values())[0].get('full_length')  # w could perhaps also just access full_length and shadowed_length in this way?
    #if dict is not None : #apparently some edges does not have a 'full_length' attribute, and in that case the attribute value will be none. check in the data frame for none values
    #    print(dict)
    full_length = min(attr.get('full_length', 0) for attr in d.values()) #if 'full_length' value is None, then it will default to 1... keyError will return None because we use the get() method
    shadow_length = min(attr.get('shadowed_part_length', 1) for attr in d.values())
    result = (shadow_length/full_length) + (full_length/shadow_length) # we get many 1´s, which either because they default to 1, or because shadow_length==full_length(very unlikely). So in the next line, we filter out all the 1´s to make sure that we actually have real meaningfull values
    #if not math.isclose(result, 1): #
    #    print(result)

    full_length = min(attr.get('full_length', 0) for attr in d.values()) 
    print("scr = ",end='') 
    print(u)    
    print("dst = ",end='') 
    print(v)
    print("full_length = ",end='') 
    print(full_length)
    print("shadow_length = ",end='')
    print(shadow_length)
    print("cost = ",end='')
    print(result)
    print()
    return result

In [132]:
def weight_function_2(u, v, d):
    #print(d.values())
    assert len(list(attr.get('full_length', 0) for attr in d.values())) == 1 #the graph only have one sttribute with name = 'full_length'
    dict = list(d.values())[0].get('full_length')  # w could perhaps also just access full_length and shadowed_length in this way?
    #if dict is not None : #apparently some edges does not have a 'full_length' attribute, and in that case the attribute value will be none. check in the data frame for none values
    #    print(dict)
    full_length = min(attr.get('full_length', 0) for attr in d.values()) #if 'full_length' value is None, then it will default to 1... keyError will return None because we use the get() method
    shadow_length = min(attr.get('shadowed_part_length', 0) for attr in d.values())
    sunny_length = full_length-shadow_length
    result = (full_length*0.8) + (sunny_length*1.2) # Applying statistical weights to sunny length and full length
    #if not math.isclose(result, 1): #
    #    print(result)

    print("scr = ",end='') 
    print(u)    
    print("dst = ",end='') 
    print(v)
    print("full_length = ",end='') 
    print(full_length)
    print("shadow_length = ",end='')
    print(shadow_length)
    print("cost = ",end='')
    print(result)
    print()

    return result

In [108]:
optimal_path_test1 = nx.shortest_path(G, source=10431, target=8229, weight=weight_function_1)

scr = 10431
dst = 4745
full_length = 83.28501899060669
shadow_length = 1
result = 83.29702595190801

scr = 10431
dst = 10846
full_length = 83.28501899060669
shadow_length = 1
result = 83.29702595190801

scr = 10431
dst = 11248
full_length = 97.84879611193904
shadow_length = 29.76836616725353
result = 3.591234161001484

scr = 6960
dst = 8229
full_length = 114.32415382569876
shadow_length = 113.05321883756268
result = 2.000124975747738

scr = 22259
dst = 8229
full_length = 64.69639567911136
shadow_length = 38.40895261145818
result = 2.2780892108155353

scr = 24525
dst = 8229
full_length = 82.48018605483345
shadow_length = 49.06954680802431
result = 2.2758086555365793

scr = 11248
dst = 10175
full_length = 83.28501899060669
shadow_length = 1
result = 83.29702595190801

scr = 11248
dst = 10431
full_length = 97.84879611193904
shadow_length = 29.76836616725353
result = 3.591234161001484

scr = 11248
dst = 19210
full_length = 83.28501899060669
shadow_length = 1
result = 83.29702595190801

scr

In [109]:
print(optimal_path_test1)

[10431, 11248, 19210, 6088, 8847, 1095, 8638, 23137, 1205, 12715, 256, 18577, 4660, 624, 8915, 2025, 19856, 8222, 11524, 14331, 5142, 11616, 21113, 17228, 17414, 17156, 15624, 10837, 3428, 11170, 5317, 22847, 6002, 20371, 24132, 3158, 1562, 6433, 24639, 8097, 4185, 19635, 922, 3266, 22043, 22169, 16917, 23159, 1624, 16602, 16299, 9320, 19406, 5046, 8794, 2815, 864, 23457, 2115, 24143, 23964, 24525, 8229]


In [133]:
optimal_path_test2 = nx.shortest_path(G, source=10431, target=8229, weight=weight_function_2)

scr = 10431
dst = 4745
full_length = 83.28501899060669
shadow_length = 0
cost = 166.57003798121337

scr = 10431
dst = 10846
full_length = 83.28501899060669
shadow_length = 0
cost = 166.57003798121337

scr = 10431
dst = 11248
full_length = 97.84879611193904
shadow_length = 29.76836616725353
cost = 159.97555282317384

scr = 6960
dst = 8229
full_length = 114.32415382569876
shadow_length = 113.05321883756268
cost = 92.9844450463223

scr = 22259
dst = 8229
full_length = 64.69639567911136
shadow_length = 38.40895261145818
cost = 83.30204822447291

scr = 24525
dst = 8229
full_length = 82.48018605483345
shadow_length = 49.06954680802431
cost = 106.07691594003774

scr = 11248
dst = 10175
full_length = 83.28501899060669
shadow_length = 0
cost = 166.57003798121337

scr = 11248
dst = 10431
full_length = 97.84879611193904
shadow_length = 29.76836616725353
cost = 159.97555282317384

scr = 11248
dst = 19210
full_length = 83.28501899060669
shadow_length = 0
cost = 166.57003798121337

scr = 7813
dst = 

In [122]:
print(optimal_path_test2)

[10431, 11248, 19210, 6088, 8847, 1095, 8638, 23137, 1205, 12715, 256, 18577, 4660, 624, 8915, 2025, 19856, 8222, 11524, 14331, 5142, 11616, 21113, 17228, 17414, 17156, 4934, 5241, 19163, 18835, 15580, 3275, 15822, 3030, 4695, 9251, 18777, 9088, 146, 21280, 18795, 4587, 11222, 17446, 2791, 14428, 25080, 12505, 22043, 22169, 16917, 23159, 1624, 16602, 16299, 9320, 19406, 5046, 8794, 2815, 864, 23457, 2115, 24143, 23964, 24525, 8229]


In [110]:
regular_shortest_path = nx.shortest_path(G, source=10431, target=8229, weight='full_length')

In [111]:
print(regular_shortest_path)

[10431, 11248, 19210, 6088, 8847, 1095, 8638, 23137, 1205, 12715, 256, 18577, 4660, 624, 8915, 2025, 19856, 8222, 11524, 14331, 5142, 11616, 21113, 17228, 21316, 9153, 540, 3428, 11170, 5317, 22847, 6002, 20371, 24132, 3158, 10449, 1002, 4374, 1890, 6785, 6054, 14428, 25080, 12505, 22043, 22169, 16917, 23159, 1624, 16602, 16299, 9320, 19406, 5046, 8794, 2815, 864, 23457, 2115, 24143, 23964, 24525, 8229]


In [ ]:
#check if path is simple
print(nx.is_simple_path(G,nx.shortest_path(G, source=9, target=15987, weight='full_length')))

In [ ]:
# A way to iterate through a Python generator object
        #count = 0
        #for item in simple_paths_within_range_generator:
        #    if count>1: 
        #        break
        #    print(item)
        #    count+=1

In [ ]:
# Graph inspection. Display the NodeDataView of the graph

# G.nodes.data(True)

In [ ]:
# Graph inspection. Display the MultiEdgeDataView of the graph

# G.edges.data(True)

In [25]:
# Extracting the columns and attributes of the edges of the Graph (Code stolen from osmnx/utils_graph.py:66)

u, v, k, data = zip(*G.edges(keys=True, data=True)) # u=edges.scr_id-->nodes.vertex_id, v=edges.dst_id-->nodes.vertex_id, k=edges.osm_id, data=edges._attribute_columns

# print(type(u)) # edges.src_id, type=tuple... The values of this tuple each represents the source node of a given edge(edges.src_id), and corresponds to a 'vertex_id'-value in the nodes data(nodes.vertex_id). However, the osmnx, and Folium libraries expect it to correspond to an osm id(nodes.osm_id) 
# print(type(v)) # edges.dst_id, type=tuple... The values of this tuple each represents the destination node of a given edge(edges.dst_id), and corresponds to a 'vertex_id'-value in the nodes data(nodes.vertex_id). However, the osmnx, and Folium libraries expect it to correspond to an osm id(nodes.osm_id)

In [24]:
# Graph inspection/extraction

# Extracting the a dictionary with the nodes_index as the key, and the chosen attribute (second argument of the function) as the value

x_lookup = nx.get_node_attributes(G, "x") # type=dict, The keys of this dict corresponds to the osm id (nodes.osm_id) of a given node(nodes.osm_id). Its value corresponds to its lon-value(nodes.lon)
y_lookup = nx.get_node_attributes(G, "y") # type=dict, The keys of this dict corresponds to the osm id (nodes.osm_id) of a given node(nodes.osm_id). Its value corresponds to its lat-value(nodes.lat)

In [32]:
# Graph inspection/extraction

edge_lookup = nx.get_edge_attributes(G, "full_length") # osm_id, scr_id and dst_id required to look up values in this dictionary. Can perhaps be used to verify the full_length values extracted in the benchmark functionality

    # ATTENTION! When expecting the edges dictionaries, like the one above, it seems that there exist an exact duplicate of each edge. At least when expecting the edge dictionary for the shadowed_part_length attribute

    # Example of how provide the index of and edge as the key to look up in the an edge dictionary:
        # edge_lookup[dst_id,src_id,osm_id] ---> e.g. edge_lookup[8983, 15939, 24454558.0])

    # Display the entire dictionary: print(edge_lookup)

In [33]:
print(edge_lookup[8983, 15939, 24454558.0])

9.029045785468297


In [71]:
#Graph inspection/extraction

# Extracting the attributes of a given edge(u,v) only by providing scr_id and dst_id. The edge data is given as dict. Afterwards specifying which attribute value to get.
edge_data = G.get_edge_data(8983,15939) #ATTENTION! 3rd argument of this function is what should be returned in case edge(u,v) is not found. It defaults to a None object, and thus we put this statement i a try statement to handle such a case
#print(edge_data)
try:
     print(min(attr.get('full_length', 0) for attr in edge_data.values())) # Should the value of the specified attribute be None, then the returned value will default to 0
     print(min(attr.get('shadowed_part_length', 0) for attr in edge_data.values())) # Should the value of the specified attribute be None, then the returned value will default to 0

except AttributeError: 
    print("edge with given scr and dst does not exist in data set")

9.029045785468297
1.7582626870419125


In [44]:
#Graph inspection/extraction

# Extracting the attributes of a given edge(u,v) only by providing scr_id and dst_id. The edge data is given as dict. Afterwards specifying which attribute value to get.
def get_src_dst_distance(src, dst):
     edge_data = G.get_edge_data(src,dst) #ATTENTION! 3rd argument of this function is what should be returned in case edge(u,v) is not found. It defaults to a None object, and thus we put this statement i a try statement to handle such a case
     #print(edge_data)
     try:
          return (min(attr.get('full_length', 0) for attr in edge_data.values())) # Should the value of the specified attribute be None, then the returned value will default to 0
     except AttributeError: 
         print("edge with given scr and dst does not exist in data set")

In [45]:
#Graph inspection/extraction

# Extracting the attributes of a given edge(u,v) only by providing scr_id and dst_id. The edge data is given as dict. Afterwards specifying which attribute value to get.
def get_src_dst_shadowed_distance(src, dst):
     edge_data = G.get_edge_data(src,dst) #ATTENTION! 3rd argument of this function is what should be returned in case edge(u,v) is not found. It defaults to a None object, and thus we put this statement i a try statement to handle such a case
     #print(edge_data)
     try:
          return (min(attr.get('shadowed_part_length', 0) for attr in edge_data.values())) # Should the value of the specified attribute be None, then the returned value will default to 0
     except AttributeError: 
         print("edge with given scr and dst does not exist in data set")

In [ ]:
print(edge_data)

In [ ]:
test_list = [9, 21096, 4847, 16077, 13009, 7958, 1809, 13392, 4021, 3745, 24909, 18872]

In [46]:
# Function for getting the total length of a path

def get_total_path_length(path):
    try:
        total_length = 0
        for index,scr in enumerate(path):
            if(index==len(path)-1):
                break
            dst = path[index+1]
            scr_dst_distance = get_src_dst_distance(scr, dst)
            total_length += scr_dst_distance
        return total_length
    except TypeError:
        print("No distance between given scr and dst")


In [47]:
# Testing

# Function for getting the total length of a path, which is covered by shadow

def get_total_path_shadowed_length(path):
    try:
        total_length = 0
        for index,scr in enumerate(path):
            if(index==len(path)-1):
                break
            dst = path[index+1]
            scr_dst_distance = get_src_dst_shadowed_distance(scr, dst)
            total_length += scr_dst_distance
        return total_length
    except TypeError:
        print("No distance between given scr and dst")


In [ ]:
get_total_path_shadowed_length(optimal_path_test1)

In [ ]:
get_total_path_shadowed_length(regular_shortest_path)

In [48]:
# Testing 

# Function for getting the fraction of a path which is covered by shadow

def get_shadowed_part_of_path(path):
    try:
        shadowed_part = get_total_path_shadowed_length(path) / get_total_path_length(path)
        return shadowed_part
    except TypeError:
        print("No distance between given scr and dst")


In [ ]:
get_shadowed_part_of_path(optimal_path_test1)

In [ ]:
get_shadowed_part_of_path(regular_shortest_path)

In [107]:
#Testing

# Function for benchmarking a cost function against the regular shortst path with 'full_length' as the weight

def cost_function_performance(src, dst, cost_function):
    regular_shortest_path = nx.shortest_path(G, source=src, target=dst, weight='full_length')
    print("INSIDE COST FUNCTION")
    print()
    optimized_path = nx.shortest_path(G, source=src, target=dst, weight=cost_function)
    print()
    print("OUTSIDE COST FUNCTION")
    print()
    shadowed_part_regular = get_shadowed_part_of_path(regular_shortest_path)
    shadowed_part_optimized = get_shadowed_part_of_path(optimized_path)
    benchmark = shadowed_part_regular
    performance = 100*(shadowed_part_optimized/benchmark)
    print("shadowed_part_regular = ",end='') 
    print(shadowed_part_regular)
    print("shadowed_part_optimized = ",end='')
    print(shadowed_part_optimized)
    print("benchmark = ", end='')
    print(benchmark)
    print("performance = ",end='')
    print(performance)
    return performance


In [55]:
get_total_path_length(optimal_path_test1)

7663.939979643108

In [62]:
get_total_path_length(regular_shortest_path)

7141.679874262998

In [105]:
print(cost_function_performance(10431,8229, weight_function_1))

INSIDE FUNCTION STARTS

scr = 10431
dst = 4745
full_length = 83.28501899060669
shadow_length = 1
result = 83.29702595190801

scr = 10431
dst = 10846
full_length = 83.28501899060669
shadow_length = 1
result = 83.29702595190801

scr = 10431
dst = 11248
full_length = 97.84879611193904
shadow_length = 29.76836616725353
result = 3.591234161001484

scr = 6960
dst = 8229
full_length = 114.32415382569876
shadow_length = 113.05321883756268
result = 2.000124975747738

scr = 22259
dst = 8229
full_length = 64.69639567911136
shadow_length = 38.40895261145818
result = 2.2780892108155353

scr = 24525
dst = 8229
full_length = 82.48018605483345
shadow_length = 49.06954680802431
result = 2.2758086555365793

scr = 11248
dst = 10175
full_length = 83.28501899060669
shadow_length = 1
result = 83.29702595190801

scr = 11248
dst = 10431
full_length = 97.84879611193904
shadow_length = 29.76836616725353
result = 3.591234161001484

scr = 11248
dst = 19210
full_length = 83.28501899060669
shadow_length = 1
result 

In [106]:
print(cost_function_performance(10431,8229, weight_function_2))

INSIDE FUNCTION STARTS

scr = 10431
dst = 4745
full_length = 83.28501899060669
shadow_length = 0
result = 166.57003798121337

scr = 10431
dst = 10846
full_length = 83.28501899060669
shadow_length = 0
result = 166.57003798121337

scr = 10431
dst = 11248
full_length = 97.84879611193904
shadow_length = 29.76836616725353
result = 159.97555282317384

scr = 6960
dst = 8229
full_length = 114.32415382569876
shadow_length = 113.05321883756268
result = 92.9844450463223

scr = 22259
dst = 8229
full_length = 64.69639567911136
shadow_length = 38.40895261145818
result = 83.30204822447291

scr = 24525
dst = 8229
full_length = 82.48018605483345
shadow_length = 49.06954680802431
result = 106.07691594003774

scr = 11248
dst = 10175
full_length = 83.28501899060669
shadow_length = 0
result = 166.57003798121337

scr = 11248
dst = 10431
full_length = 97.84879611193904
shadow_length = 29.76836616725353
result = 159.97555282317384

scr = 11248
dst = 19210
full_length = 83.28501899060669
shadow_length = 0
resu

In [ ]:
#########################################################################################################################################################################################################

In [ ]:
# This is a function to get the difference in number of nodes for two paths ---> difference in path degree

In [63]:
def get_degree_difference(path1, path2):
    return len(path1)-len(path2)


In [120]:
print(get_degree_difference(optimal_path_test1,regular_shortest_path))

0


In [ ]:
#This is a function to get number of nodes that differ between two paths

In [114]:
def some_function_name(path1, path2):
    difference_count = 0
    for node1 in path1:
        for node2 in path2:
            if node1==node2:
                difference_count +=1
    return difference_count

In [115]:
print(some_function_name(optimal_path_test1,regular_shortest_path))

51


In [123]:
print(some_function_name(optimal_path_test2,regular_shortest_path))

46


In [116]:
print(len(optimal_path_test1))

63


In [124]:
print(len(optimal_path_test2))

67


In [117]:
print(len(regular_shortest_path))

63


In [118]:
get_total_path_length(optimal_path_test1)

5215.36419615842

In [125]:
get_total_path_length(optimal_path_test2)

5085.922572036992

In [119]:
get_total_path_length(regular_shortest_path)

4990.062772557522

In [ ]:
#########################################################################################################################################################################################################

In [36]:
p_u = Point((x_lookup[15633],y_lookup[15633]))
p_v = Point((x_lookup[1],y_lookup[1]))

In [30]:
x_lookup[]
y_lookup[]

SyntaxError: invalid syntax (4267951951.py, line 1)

In [27]:
print(p_u)
print(p_v)

POINT (-1.2338981 52.9724785)
POINT (-1.0771582 53.0405631)


In [43]:
string = LineString([(-1.2338981,52.9724785),(-1.0771582,53.0405631)])

In [40]:
print (string)

{"coordinates": [[-1.233898, 52.972479], [-1.077158, 53.040563]], "type": "LineString"}


In [42]:
LineString([p_u,p_v])

ValueError: <shapely.geometry.point.Point object at 0x7f538c59d660> is not a JSON compliant number

In [ ]:
# Function for extracting the LineString of the shortest path

def get_path_coords(path):
    x_lookup = nx.get_node_attributes(G, "x")
    y_lookup = nx.get_node_attributes(G, "y")
    
    path_nodes_coords = []
    for node in path:
        node_coords = [x_lookup[node],y_lookup[node]]
        path_nodes_coords.append(node_coords)
    return path_nodes_coords

In [ ]:
get_path_coords(test)

In [ ]:
# Function which makes feature collection from a list
def coords_to_geoJson(listCoord):
    coords = []
    for pt in listCoord:
         coords.append((pt[0], pt[1]))


    my_list = LineString(coords)

    my_feature = Feature(geometry=my_list)

    feature_collection = FeatureCollection([my_feature])
    return feature_collection

In [41]:
ox.folium.plot_graph_folium(G)
#REASON FOR FAILING: 
    # Assuming that the 'vertex_id' column has been chosen as the index of the nodes dataframe. The reason for failing is that the LineString computation in osmnx/utils_graph.py:79 fails. The reason this
    # computation fails is that the tuple of edges.src_id´s(u) and the tuple of edges.dst_id´s (v) contains values which corresponds to 'vertex_id'-values of helper nodes. Since helper nodes have been removed, the x_lookup and 
    # y_lookup does not contain vertex_id of these nodes as keys, and thus we get a KeyErro when trying to look them up. Important to note that the edges that have these non-existing helper nodes as either their source or 
    # destination, are not themselves helper-edges. They are real edges which, for some reason, are attached to a non-existing helper node in one of its ends.
    # At first glance, it seems like leaving all helper nodes be in the nodes data framwe when building the graph would be a solution. However, it could seems like either osmnx or networkX removes nodes which has no value for 
    # for their 'geopint' attribute, which is the case for the helper nodes.     

KeyError: 25262